## Bayesian methods of hyperparameter optimization

In addition to the random search and the grid search methods for selecting optimal hyperparameters, we can use Bayesian methods of probabilities to select the optimal hyperparameters for an algorithm.

In this case study, we will be using the BayesianOptimization library to perform hyperparmater tuning. This library has very good documentation which you can find here: https://github.com/fmfn/BayesianOptimization

You will need to install the Bayesian optimization module. Running a cell with an exclamation point in the beginning of the command will run it as a shell command — please do this to install this module from our notebook in the cell below.

In [1]:
! pip install bayesian-optimization

In [4]:
! pip install lightgbm

In [6]:
! pip install catboost

     -------------------------------------- 101.0/101.0 MB 4.8 MB/s eta 0:00:00


In [7]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
import lightgbm
from bayes_opt import BayesianOptimization
from catboost import CatBoostClassifier, cv, Pool

In [8]:
import os
os.listdir()

['.ipynb_checkpoints',
 'Bayesian_optimization_case_study.ipynb',
 'flight_delays_test.csv',
 'flight_delays_test.csv.zip',
 'flight_delays_train.csv',
 'flight_delays_train.csv.zip']

## How does Bayesian optimization work?

Bayesian optimization works by constructing a posterior distribution of functions (Gaussian process) that best describes the function you want to optimize. As the number of observations grows, the posterior distribution improves, and the algorithm becomes more certain of which regions in parameter space are worth exploring and which are not, as seen in the picture below.

<img src="https://github.com/fmfn/BayesianOptimization/blob/master/examples/bo_example.png?raw=true" />
As you iterate over and over, the algorithm balances its needs of exploration and exploitation while taking into account what it knows about the target function. At each step, a Gaussian Process is fitted to the known samples (points previously explored), and the posterior distribution, combined with an exploration strategy (such as UCB — aka Upper Confidence Bound), or EI (Expected Improvement). This process is used to determine the next point that should be explored (see the gif below).
<img src="https://github.com/fmfn/BayesianOptimization/raw/master/examples/bayesian_optimization.gif" />

## Let's look at a simple example

The first step is to create an optimizer. It uses two items:
* function to optimize
* bounds of parameters

The function is the procedure that counts metrics of our model quality. The important thing is that our optimization will maximize the value on function. Smaller metrics are best. Hint: don't forget to use negative metric values.

Here we define our simple function we want to optimize.

In [9]:
def simple_func(a, b):
    return a + b

Now, we define our bounds of the parameters to optimize, within the Bayesian optimizer.

In [14]:
optimizer = BayesianOptimization(
    simple_func,
    {'a': (1, 3),
    'b': (4, 7)})

These are the main parameters of this function:

* **n_iter:** This is how many steps of Bayesian optimization you want to perform. The more steps, the more likely you are to find a good maximum.

* **init_points:** This is how many steps of random exploration you want to perform. Random exploration can help by diversifying the exploration space.

Let's run an example where we use the optimizer to find the best values to maximize the target value for a and b given the inputs of 3 and 2.

In [15]:
optimizer.maximize(n_iter = 3, init_points = 2)

|   iter    |  target   |     a     |     b     |
-------------------------------------------------
| 1         | 6.339     | 2.154     | 4.185     |
| 2         | 9.014     | 2.508     | 6.506     |
| 3         | 9.025     | 2.52      | 6.505     |
| 4         | 8.0       | 1.0       | 7.0       |
| 5         | 10.0      | 3.0       | 7.0       |


Great, now let's print the best parameters and the associated maximized target.

In [16]:
print(optimizer.max['params']);optimizer.max['target']

{'a': 3.0, 'b': 7.0}


10.0

## Test it on real data using the Light GBM

The dataset we will be working with is the famous flight departures dataset. Our modeling goal will be to predict if a flight departure is going to be delayed by 15 minutes based on the other attributes in our dataset. As part of this modeling exercise, we will use Bayesian hyperparameter optimization to identify the best parameters for our model.

**<font color='teal'> You can load the zipped csv files just as you would regular csv files using Pandas read_csv. In the next cell load the train and test data into two seperate dataframes. </font>**


In [17]:
train_df = pd.read_csv('flight_delays_train.csv.zip')
test_df = pd.read_csv('flight_delays_test.csv.zip')

**<font color='teal'> Print the top five rows of the train dataframe and review the columns in the data. </font>**

In [18]:
train_df.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


In [19]:
test_df.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance
0,c-7,c-25,c-3,615,YV,MRY,PHX,598
1,c-4,c-17,c-2,739,WN,LAS,HOU,1235
2,c-12,c-2,c-7,651,MQ,GSP,ORD,577
3,c-3,c-25,c-7,1614,WN,BWI,MHT,377
4,c-6,c-6,c-3,1505,UA,ORD,STL,258


**<font color='teal'> Use the describe function to review the numeric columns in the train dataframe. </font>**

In [20]:
train_df.describe()

,DepTime,Distance
count,100000.000000,100000.00000
mean,1341.523880,729.39716
std,476.378445,574.61686
min,1.000000,30.00000
25%,931.000000,317.00000
50%,1330.000000,575.00000
75%,1733.000000,957.00000
max,2534.000000,4962.00000


In [21]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Month              100000 non-null  object
 1   DayofMonth         100000 non-null  object
 2   DayOfWeek          100000 non-null  object
 3   DepTime            100000 non-null  int64 
 4   UniqueCarrier      100000 non-null  object
 5   Origin             100000 non-null  object
 6   Dest               100000 non-null  object
 7   Distance           100000 non-null  int64 
 8   dep_delayed_15min  100000 non-null  object
dtypes: int64(2), object(7)
memory usage: 6.9+ MB


Notice, `DepTime` is the departure time in a numeric representation in 2400 hours. 

 **<font color='teal'>The response variable is 'dep_delayed_15min' which is a categorical column, so we need to map the Y for yes and N for no values to 1 and 0. Run the code in the next cell to do this.</font>**

In [22]:
#train_df = train_df[train_df.DepTime <= 2400].copy()
y_train = train_df['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values
type(y_train)

numpy.ndarray

In [24]:
y_train.shape

(100000,)

## Feature Engineering
Use these defined functions to create additional features for the model. Run the cell to add the functions to your workspace.

In [25]:
def label_enc(df_column):
    df_column = LabelEncoder().fit_transform(df_column)
    return df_column

def make_harmonic_features_sin(value, period=2400):
    value *= 2 * np.pi / period 
    return np.sin(value)

def make_harmonic_features_cos(value, period=2400):
    value *= 2 * np.pi / period 
    return np.cos(value)

def feature_eng(df):
    df['flight'] = df['Origin']+df['Dest']
    df['Month'] = df.Month.map(lambda x: x.split('-')[-1]).astype('int32')
    df['DayofMonth'] = df.DayofMonth.map(lambda x: x.split('-')[-1]).astype('uint8')
    df['begin_of_month'] = (df['DayofMonth'] < 10).astype('uint8')
    df['midddle_of_month'] = ((df['DayofMonth'] >= 10)&(df['DayofMonth'] < 20)).astype('uint8')
    df['end_of_month'] = (df['DayofMonth'] >= 20).astype('uint8')
    df['DayOfWeek'] = df.DayOfWeek.map(lambda x: x.split('-')[-1]).astype('uint8')
    df['hour'] = df.DepTime.map(lambda x: x/100).astype('int32')
    df['morning'] = df['hour'].map(lambda x: 1 if (x <= 11)& (x >= 7) else 0).astype('uint8')
    df['day'] = df['hour'].map(lambda x: 1 if (x >= 12) & (x <= 18) else 0).astype('uint8')
    df['evening'] = df['hour'].map(lambda x: 1 if (x >= 19) & (x <= 23) else 0).astype('uint8')
    df['night'] = df['hour'].map(lambda x: 1 if (x >= 0) & (x <= 6) else 0).astype('int32')
    df['winter'] = df['Month'].map(lambda x: x in [12, 1, 2]).astype('int32')
    df['spring'] = df['Month'].map(lambda x: x in [3, 4, 5]).astype('int32')
    df['summer'] = df['Month'].map(lambda x: x in [6, 7, 8]).astype('int32')
    df['autumn'] = df['Month'].map(lambda x: x in [9, 10, 11]).astype('int32')
    df['holiday'] = (df['DayOfWeek'] >= 5).astype(int) 
    df['weekday'] = (df['DayOfWeek'] < 5).astype(int)
    df['airport_dest_per_month'] = df.groupby(['Dest', 'Month'])['Dest'].transform('count')
    df['airport_origin_per_month'] = df.groupby(['Origin', 'Month'])['Origin'].transform('count')
    df['airport_dest_count'] = df.groupby(['Dest'])['Dest'].transform('count')
    df['airport_origin_count'] = df.groupby(['Origin'])['Origin'].transform('count')
    df['carrier_count'] = df.groupby(['UniqueCarrier'])['Dest'].transform('count')
    df['carrier_count_per month'] = df.groupby(['UniqueCarrier', 'Month'])['Dest'].transform('count')
    df['deptime_cos'] = df['DepTime'].map(make_harmonic_features_cos)
    df['deptime_sin'] = df['DepTime'].map(make_harmonic_features_sin)
    df['flightUC'] = df['flight']+df['UniqueCarrier']
    df['DestUC'] = df['Dest']+df['UniqueCarrier']
    df['OriginUC'] = df['Origin']+df['UniqueCarrier']
    return df.drop('DepTime', axis=1)

Concatenate the training and testing dataframes.


In [26]:
full_df = pd.concat([train_df.drop('dep_delayed_15min', axis=1), test_df])
full_df = feature_eng(full_df)

In [28]:
pd.set_option('display.max_columns', None)
full_df.head()

,Month,DayofMonth,DayOfWeek,UniqueCarrier,Origin,Dest,Distance,flight,begin_of_month,midddle_of_month,end_of_month,hour,morning,day,evening,night,winter,spring,summer,autumn,holiday,weekday,airport_dest_per_month,airport_origin_per_month,airport_dest_count,airport_origin_count,carrier_count,carrier_count_per month,deptime_cos,deptime_sin,flightUC,DestUC,OriginUC
0,8,21,7,AA,ATL,DFW,732,ATLDFW,0,0,1,19,0,0,1,0,0,0,1,0,1,0,746,1019,8290,11387,18024,1569,0.343660,-0.939094,ATLDFWAA,DFWAA,ATLAA
1,4,20,3,US,PIT,MCO,834,PITMCO,0,0,1,15,0,1,0,0,0,1,0,0,0,1,313,105,3523,1390,13069,1094,-0.612907,-0.790155,PITMCOUS,MCOUS,PITUS
2,9,2,5,XE,RDU,CLE,416,RDUCLE,1,0,0,14,0,1,0,0,0,0,0,1,1,0,166,136,2247,1747,11737,977,-0.835807,-0.549023,RDUCLEXE,CLEXE,RDUXE
3,11,25,6,OO,DEN,MEM,872,DENMEM,0,0,1,10,1,0,0,0,0,0,0,1,1,0,136,514,1785,6222,15343,1242,-0.884988,0.465615,DENMEMOO,MEMOO,DENOO
4,10,7,6,WN,MDW,OMA,423,MDWOMA,1,0,0,18,0,1,0,0,0,0,0,1,1,0,48,226,687,2571,30958,2674,0.073238,-0.997314,MDWOMAWN,OMAWN,MDWWN


Apply the earlier defined feature engineering functions to the full dataframe.

In [29]:
for column in ['UniqueCarrier', 'Origin', 'Dest','flight',  'flightUC', 'DestUC', 'OriginUC']:
    full_df[column] = label_enc(full_df[column])


Split the new full dataframe into X_train and X_test. 

In [30]:
X_train = full_df[:train_df.shape[0]]
X_test = full_df[train_df.shape[0]:]

In [32]:
train_df.shape, X_train.shape

((100000, 9), (100000, 33))

Create a list of the categorical features.

In [33]:
categorical_features = ['Month',  'DayOfWeek', 'UniqueCarrier', 'Origin', 'Dest','flight',  'flightUC', 'DestUC', 'OriginUC']

In [34]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 33 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Month                     100000 non-null  int32  
 1   DayofMonth                100000 non-null  uint8  
 2   DayOfWeek                 100000 non-null  uint8  
 3   UniqueCarrier             100000 non-null  int32  
 4   Origin                    100000 non-null  int32  
 5   Dest                      100000 non-null  int32  
 6   Distance                  100000 non-null  int64  
 7   flight                    100000 non-null  int32  
 8   begin_of_month            100000 non-null  uint8  
 9   midddle_of_month          100000 non-null  uint8  
 10  end_of_month              100000 non-null  uint8  
 11  hour                      100000 non-null  int32  
 12  morning                   100000 non-null  uint8  
 13  day                       100000 non-null  ui

Let's build a light GBM model to test the bayesian optimizer.

### [LightGBM](https://lightgbm.readthedocs.io/en/latest/) is a gradient boosting framework that uses tree-based learning algorithms. It is designed to be distributed and efficient with the following advantages:

* Faster training speed and higher efficiency.
* Lower memory usage.
* Better accuracy.
* Support of parallel and GPU learning.
* Capable of handling large-scale data.

First, we define the function we want to maximize and that will count cross-validation metrics of lightGBM for our parameters.

Some params such as num_leaves, max_depth, min_child_samples, min_data_in_leaf should be integers.

In [44]:
def lgb_eval(num_leaves,max_depth,lambda_l2,lambda_l1,min_child_samples, min_data_in_leaf):
    params = {
        "objective" : "binary",
        "metric" : "auc", 
        'is_unbalance': True,
        "num_leaves" : int(num_leaves),
        "max_depth" : int(max_depth),
        "lambda_l2" : lambda_l2,
        "lambda_l1" : lambda_l1,
        "num_threads" : 20,
        "min_child_samples" : int(min_child_samples),
        'min_data_in_leaf': int(min_data_in_leaf),
        "learning_rate" : 0.03,
        "subsample_freq" : 5,
        "bagging_seed" : 42,
        "verbosity" : -1
    }
    lgtrain = lightgbm.Dataset(X_train, y_train,categorical_feature=categorical_features)
    cv_result = lightgbm.cv(params,
                       lgtrain,
                       1000,
                       stratified=True,
                       nfold=3)
    return cv_result['valid auc-mean'][-1]

Apply the Bayesian optimizer to the function we created in the previous step to identify the best hyperparameters. We will run 10 iterations and set init_points = 2.


In [45]:
lgbBO = BayesianOptimization(lgb_eval, {'num_leaves': (25, 4000),
                                                'max_depth': (5, 63),
                                                'lambda_l2': (0.0, 0.05),
                                                'lambda_l1': (0.0, 0.05),
                                                'min_child_samples': (50, 10000),
                                                'min_data_in_leaf': (100, 2000)
                                                })

lgbBO.maximize(n_iter=10, init_points=2)

|   iter    |  target   | lambda_l1 | lambda_l2 | max_depth | min_ch... | min_da... | num_le... |
-------------------------------------------------------------------------------------------------
| 1         | 0.7207    | 0.0256    | 0.01385   | 28.81     | 7.779e+03 | 633.7     | 3.676e+03 |
| 2         | 0.7215    | 0.01025   | 0.03168   | 62.89     | 1.465e+03 | 714.2     | 927.0     |
| 3         | 0.7427    | 0.004709  | 0.003918  | 47.51     | 279.5     | 1.402e+03 | 3.707e+03 |
| 4         | 0.7407    | 0.01228   | 0.03265   | 5.906     | 9.178e+03 | 1.257e+03 | 2.057e+03 |
| 5         | 0.7044    | 0.03018   | 0.03268   | 16.61     | 3.076e+03 | 145.8     | 2.708e+03 |
| 6         | 0.7419    | 0.03523   | 0.04877   | 45.94     | 388.8     | 1.225e+03 | 1.001e+03 |
| 7         | 0.7433    | 0.008775  | 0.005023  | 17.06     | 4.06e+03  | 1.802e+03 | 2.65e+03  |
| 8         | 0.7417    | 0.03897   | 0.04983   | 45.71     | 3.346e+03 | 1.208e+03 | 2.586e+03 |
| 9         | 0.7433

 **<font color='teal'> Print the best result by using the '.max' function.</font>**

In [46]:
lgbBO.max

{'target': 0.7435575588664315,
 'params': {'lambda_l1': 0.006365982223977823,
  'lambda_l2': 0.006740719110030752,
  'max_depth': 30.18020173510521,
  'min_child_samples': 333.7200917064581,
  'min_data_in_leaf': 1452.9742042765374,
  'num_leaves': 3681.595001261425}}

Review the process at each step by using the '.res[0]' function.

In [47]:
lgbBO.res

[{'target': 0.7207260203817037,
  'params': {'lambda_l1': 0.02559680586233308,
   'lambda_l2': 0.01384743904757837,
   'max_depth': 28.80847022336075,
   'min_child_samples': 7778.6740512688875,
   'min_data_in_leaf': 633.6966040535548,
   'num_leaves': 3675.9034938533327}},
 {'target': 0.7214527701248779,
  'params': {'lambda_l1': 0.010249168498752449,
   'lambda_l2': 0.031679566660886345,
   'max_depth': 62.893847552378716,
   'min_child_samples': 1465.348982633972,
   'min_data_in_leaf': 714.1692087972928,
   'num_leaves': 927.0474265978855}},
 {'target': 0.7426548817790888,
  'params': {'lambda_l1': 0.004709405330015449,
   'lambda_l2': 0.003918097602313242,
   'max_depth': 47.51107359046027,
   'min_child_samples': 279.46272705049597,
   'min_data_in_leaf': 1401.8024823970354,
   'num_leaves': 3707.401162618474}},
 {'target': 0.7406723111329474,
  'params': {'lambda_l1': 0.012275168427623684,
   'lambda_l2': 0.03265283011059572,
   'max_depth': 5.9057241061557155,
   'min_child_sa

In [50]:
params = {
        "objective" : "binary",
        "metric" : "auc", 
        'is_unbalance': True,
        "num_leaves" : int(lgbBO.max['params']['num_leaves']),
        "max_depth" : int(lgbBO.max['params']['max_depth']),
        "lambda_l2" : lgbBO.max['params']['lambda_l2'],
        "lambda_l1" : lgbBO.max['params']['lambda_l1'],
        "num_threads" : 20,
        "min_child_samples" : int(lgbBO.max['params']['min_child_samples']),
        'min_data_in_leaf': int(lgbBO.max['params']['min_data_in_leaf']),
        "learning_rate" : 0.03,
        "subsample_freq" : 5,
        "bagging_seed" : 42,
        "verbosity" : -1}

lgtrain = lightgbm.Dataset(X_train, y_train,categorical_feature=categorical_features)
bst = lightgbm.train(params, lgtrain, 1000)
y_pred = bst.predict(X_test)

In [51]:
y_pred.shape, y_pred[:5]

((100000,),
 array([0.07445743, 0.19766139, 0.0405125 , 0.48305038, 0.605362  ]))